In [2]:
import pysam
from collections import defaultdict
import copy

In [1]:
inBam= pysam.Samfile("/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/Trp53_processed.usassembled.bam", "rb")
outBam= pysam.Samfile("/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/Trp53_processed.unassembled.consensusTest.bam", "wb", template = inBam )
myData = {}
#in the header of the bam, only the barcode was saved...

count = 0
bamEntry = inBam.fetch( until_eof = True )
for line in bamEntry:
    count +=1
    myCigar = str(line.cigar)
    if myCigar == 'None':
        continue
    if line.is_read1 is True:
        tag = str("%s%s%s" % (line.qname[2:-2],'-','R1'))
    if line.is_read2 is True:
        tag = str("%s%s%s" % (line.qname[2:-2],'-','R2'))
    if(tag not in myData):
        myData[tag] = {count:{"cigar": line.cigar, "sequence":line.seq, "flag":line.flag, "rname": line.rname,"pos":line.pos,"mpos":line.mpos,"isize":line.isize, "mrnm":line.mrnm, "qual":line.qual }}
    else:
        myData[tag][count] = {"cigar": line.cigar, "sequence":line.seq, "flag":line.flag, "rname": line.rname,"pos":line.pos,"mpos":line.mpos,"isize":line.isize, "mrnm":line.mrnm, "qual":line.qual}

NameError: name 'pysam' is not defined

In [13]:
outBam= pysam.Samfile("/home/moetjens/kidd-lab-scratch/moetjens-projects/bsmn/MIPs/Run_2015-03-06/Trp53_processed.unassembled.consensusTest.bam", "wb", template=inBam)

for tag in myData.keys():
    a=pysam.AlignedRead()
    consensusSeq = ""
    mySequences = [] #use these lists to get the common descriptor from each tag with max command
    myCigar = []
    myPos = []
    myMpos= []
    myMrnm= []
    myIsize= []
    myRname = []
    myFlag = []
    myQual= []
    commonPos= ""#get the most common positon
    commonMpos=""
    commonMrnm=""
    commonIsize=""
    commonRname=""
    commonFlag=""
    commonQual=""
    barcodeCount = len(myData[tag]) # number of tag in a family
    count = 0 # count to keep track of indexs sorted through
    if barcodeCount > 2:
        for index in myData[tag].keys(): # iterate through all the reads of the current tag
            count +=1
            if len(myData[tag][index]['cigar']) > 1:
                continue
            myCigar.append(myData[tag][index]['cigar']) 
            myPos.append(myData[tag][index]['pos'])
            myMpos.append(myData[tag][index]['mpos'])
            myMrnm.append(myData[tag][index]['mrnm'])
            myIsize.append(myData[tag][index]['isize'])
            myRname.append(myData[tag][index]['rname'])
            myFlag.append(myData[tag][index]['flag'])
            myQual.append(myData[tag][index]['qual'])
            if barcodeCount == count: #when finished sorting through the index
                commonCigar=max(myCigar) #get the most common cigar tuple
                commonPos=max(myPos) #get the most common positon
                commonMpos=max(myMpos)
                commonMrnm=max(myMrnm)
                commonIsize=max(myIsize)
                commonRname=max(myRname)
                commonFlag=max(myFlag)
                commonQual=max(myQual)
                #print commonRname
                for index in myData[tag].keys():
                    if myData[tag][index]['cigar']==commonCigar and myData[tag][index]['pos']==commonPos : #take sequences that have the common position and cigar string
                        mySequences.append(myData[tag][index]['sequence'])
                familySize = len(mySequences)
                if familySize > 2:
                    consensusSeq = ""
                    for pos in range(0, len(mySequences[0])):
                        consensus_list=[]
                        for seq in mySequences:
                            consensus_list.append(seq[pos])
                        consensusSeq += max(consensus_list)
    if consensusSeq and consensusSeq.count("N")/ len(consensusSeq) <= 0.3: 
        a = pysam.AlignedRead()
        a.qname = str(tag) + ":" + str(familySize)
        #a.flag = 8 #mate unmapped
        a.flag = commonFlag
        a.seq = consensusSeq
        a.rname = commonRname
        a.pos = commonPos
        a.mapq = 255
        a.cigar = commonCigar
        a.mrnm = commonMrnm
        a.mpos= commonMpos
        a.isize = 0
        a.qual = 'J' * len(consensusSeq)
        outBam.write(a)

outBam.close()

In [10]:
print a.cigar
len(a.cigar)

[(0, 22), (4, 54)]


2

In [8]:
print line

['CGTAGAGAAGA',	131	10	69591296	50	[(4, 41), (0, 23)]	10	69591297	64	CGAGATCTACACATACGAGATCCGTAATCGGGAAGCTGAAGCCAAACTGCTAGCTCCCATCACC	FFEC@DGGFECF@EGFFFF@@CFDEF7GCFGGGGGGFF8DGGGGFFDECGFFCGCGEGGEF@AF	[('NM', 1), ('AS', 22), ('XS', 19)]
